In [1]:
import numpy as np
import pandas as pd
from nltk.metrics import edit_distance

In [2]:
# reading csv of all candidates declared as rep or dem
candidate_df = pd.read_csv("candidatelist.csv", header=2)

In [3]:
candidate_df.head()

,First Name,Last Name,Address 1,Address 2,City,Zip,DO,Office Title,Pseudo,District,...,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Unnamed: 20,Office ID
0,Maria,McLaughlin,Philadelphia County,1901 John F Kennedy Blvd,Philadelphia,19103,1,Justice of the Supreme Court,1,COUNTYWIDE,...,False,101,0,1,10,1,COUNTYWIDE,COUNTY,NaN,130
1,Paula,Patrick,Philadelphia County,7017 Sherwood Rd,Philadelphia,19151,1,Justice of the Supreme Court,1,COUNTYWIDE,...,False,101,0,1,10,1,COUNTYWIDE,COUNTY,NaN,130
2,Kevin,Brobson,Dauphin County,3400 Hickory Hollow Rd,Harrisburg,17112,2,Justice of the Supreme Court,1,COUNTYWIDE,...,False,101,0,1,10,1,COUNTYWIDE,COUNTY,NaN,130
3,Patricia A.,McCullough,Allegheny County,2701 Bingham Drive,Pittsburgh,15241,3,Justice of the Supreme Court,1223,COUNTYWIDE,...,False,215,4,4,10,1,COUNTYWIDE,COUNTY,NaN,130
4,Jill,Beck,Allegheny County,5342 Pocusset Street,Pittsburgh,15217,1,Judge of the Superior Court,781,COUNTYWIDE,...,False,188,14,37,10,1,COUNTYWIDE,COUNTY,NaN,140


In [4]:
pgh_districts = [x for x in candidate_df["District"].unique() if "PITTSBURGH" in x and "EAST" not in x and "E" not in x]

In [5]:
candidate_df["Office Title"].value_counts()

School Director                       504
Member of Council                     466
Judge of Election                     243
Inspector of Election                 181
Mayor                                 116
Constable                             116
Tax Collector                          96
Commissioner                           79
Magisterial District Judge             72
Judge of the Court of Common Pleas     67
Supervisor                             36
School Director At-Large               14
Member of County Council               11
Auditor                                 7
Judge of the Commonwealth Court         6
Member of Council At-Large              6
Judge of the Superior Court             4
Justice of the Supreme Court            4
Commissioner At-Large                   4
Sheriff                                 2
Controller                              2
Treasurer                               1
Name: Office Title, dtype: int64

In [6]:
candidate_df[candidate_df["Office Title"].isin(["School Director","School Director At-Large"])]

,First Name,Last Name,Address 1,Address 2,City,Zip,DO,Office Title,Pseudo,District,...,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Unnamed: 20,Office ID
104,Carlos A.,Thomas,NaN,7152 Upland St.,Pittsburgh,15208,1,School Director,729,DISTRICT 1,...,False,188,13,4,4,1,CITY SCHOOLS,PGHSCH1,NaN,710
105,Sylvia C.,Wilson,NaN,P.O.Box 4812,Pittsburgh,15206,2,School Director,714,DISTRICT 1,...,False,188,12,5,4,1,CITY SCHOOLS,PGHSCH1,NaN,710
106,Grace,Higginbotham,NaN,7103 Kedron St.,Pittsburgh,15208,3,School Director,726,DISTRICT 1,...,False,188,13,1,4,1,CITY SCHOOLS,PGHSCH1,NaN,710
107,Sylvia C.,Wilson,NaN,P.O.Box 4812,Pittsburgh,15206,1,School Director,714,DISTRICT 1,...,False,188,12,5,4,1,CITY SCHOOLS,PGHSCH1,NaN,710
108,Lamont,"Frazier, Jr.",NaN,5161 Hillcrest St.,Pittsburgh,15224,1,School Director,691,DISTRICT 3,...,False,188,10,19,4,1,CITY SCHOOLS,PGHSCH3,NaN,710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689,Todd,Kosenina,NaN,47 Trotter Ln.,Clinton,15026,2,School Director,204,WEST ALLEGHENY REGION 3,...,False,141,0,1,4,1,SCH/REGION,SCH0070,NaN,1000
690,Steven Russell,Pettit,NaN,420 Colt Cir,Clinton,15026,1,School Director,206,WEST ALLEGHENY REGION 3,...,False,141,0,3,4,1,SCH/REGION,SCH0070,NaN,1000
691,Todd,Kosenina,NaN,47 Trotter Ln.,Clinton,15026,2,School Director,204,WEST ALLEGHENY REGION 3,...,False,141,0,1,4,1,SCH/REGION,SCH0070,NaN,1000
692,Charles B.,Wade,NaN,"531 Park Ave,",Clariton,15025,1,School Director,126,CLAIRTON REGION 4,...,False,123,4,2,4,1,SCH/REGION,SCH0012,NaN,1010


# Taking a look at School Directors
## Data Cleaning

In [7]:
school_df = candidate_df[candidate_df["Office Title"].isin(["School Director","School Director At-Large"])]

In [8]:
#lets not bother w pittsburgh
school_df = school_df[school_df["District"].isin(pgh_districts) == False]

In [9]:
school_df["District"].value_counts()

PLUM                       26
MCKEESPORT AREA            24
MOON AREA                  22
GATEWAY                    21
MT LEBANON                 20
                           ..
STO-ROX REGION 1            1
WOODLAND HILLS REGION 2     1
EAST ALLEGHENY REGION 1     1
CLAIRTON REGION 4           1
CLAIRTON REGION 3           1
Name: District, Length: 72, dtype: int64

In [10]:
# this is way too many columns
school_df.head()

,First Name,Last Name,Address 1,Address 2,City,Zip,DO,Office Title,Pseudo,District,...,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Unnamed: 20,Office ID
104,Carlos A.,Thomas,NaN,7152 Upland St.,Pittsburgh,15208,1,School Director,729,DISTRICT 1,...,False,188,13,4,4,1,CITY SCHOOLS,PGHSCH1,NaN,710
105,Sylvia C.,Wilson,NaN,P.O.Box 4812,Pittsburgh,15206,2,School Director,714,DISTRICT 1,...,False,188,12,5,4,1,CITY SCHOOLS,PGHSCH1,NaN,710
106,Grace,Higginbotham,NaN,7103 Kedron St.,Pittsburgh,15208,3,School Director,726,DISTRICT 1,...,False,188,13,1,4,1,CITY SCHOOLS,PGHSCH1,NaN,710
107,Sylvia C.,Wilson,NaN,P.O.Box 4812,Pittsburgh,15206,1,School Director,714,DISTRICT 1,...,False,188,12,5,4,1,CITY SCHOOLS,PGHSCH1,NaN,710
108,Lamont,"Frazier, Jr.",NaN,5161 Hillcrest St.,Pittsburgh,15224,1,School Director,691,DISTRICT 3,...,False,188,10,19,4,1,CITY SCHOOLS,PGHSCH3,NaN,710


In [11]:
school_indices = list(school_df.index)

In [12]:
school_cleaned = pd.DataFrame()

In [13]:
name = []
party = []
district = []
for i in school_indices:
    row = school_df[school_df.index==i]
    name.append(row["First Name"].to_string(index=False) + " " + row["Last Name"].to_string(index=False))
    party.append(row["Party"].to_string(index=False))
    district.append(row["District"].to_string(index=False))

In [14]:
district2 = []
ward = []
for x in district:
    if "REGION" in x:
        parts = x.split("REGION")
        ward.append(int(parts[-1]))
        district2.append(' '.join(parts[:-1]).strip())
    else:
        ward.append(np.nan)
        district2.append(x)
    

In [15]:
district3 = []
for x in district2:
    if x == "PINE RICHLAND":
        district3.append("PINE-RICHLAND")
    else:
        district3.append(x)

In [16]:
school_cleaned["name"] = name
school_cleaned["party"] = party
school_cleaned["district"] = district3
school_cleaned["ward"] = ward

In [17]:
school_cleaned.head()

,name,party,district,ward
0,Carlos A. Thomas,Democratic,DISTRICT 1,NaN
1,Sylvia C. Wilson,Democratic,DISTRICT 1,NaN
2,Grace Higginbotham,Democratic,DISTRICT 1,NaN
3,Sylvia C. Wilson,Republican,DISTRICT 1,NaN
4,"Lamont Frazier, Jr.",Democratic,DISTRICT 3,NaN


In [18]:
school_pos_df = pd.read_csv("schoolpos.csv")

In [19]:
school_pos_df.columns = ["num","district","term","ward"]

In [20]:
school_pos_df.head()

,num,district,term,ward
0,4,ALLEGHENY VALLEY,4-year,NaN
1,4,AVONWORTH,4-year,NaN
2,4,BALDWIN-WHITEHALL,4-year,NaN
3,4,BETHEL PARK,4-year,NaN
4,4,BRENTWOOD,4-year,NaN


In [21]:
school_pos_df.term = school_pos_df.term.map(lambda x: int(x.split("-")[0]))

In [22]:
school_pos_df.head()

,num,district,term,ward
0,4,ALLEGHENY VALLEY,4,NaN
1,4,AVONWORTH,4,NaN
2,4,BALDWIN-WHITEHALL,4,NaN
3,4,BETHEL PARK,4,NaN
4,4,BRENTWOOD,4,NaN


In [23]:
school_pos_cleaned = school_pos_df[school_pos_df.ward.notna()==False]

In [24]:
school_pos_cleaned.head()

,num,district,term,ward
0,4,ALLEGHENY VALLEY,4,NaN
1,4,AVONWORTH,4,NaN
2,4,BALDWIN-WHITEHALL,4,NaN
3,4,BETHEL PARK,4,NaN
4,4,BRENTWOOD,4,NaN


In [25]:
schoolward_df = school_pos_df[school_pos_df.ward.notna()==True]

In [26]:
def dewardify(ward_df):
    row_list = []
    for row in range(len(ward_df)):
        regions = ward_df.iloc[[row]].ward.values[0].split(",")
        num = int(ward_df.iloc[row,0])//len(regions)
        if len(regions)>1:
            for x in regions:
                temp_dict = {
                    "num": num, 
                    "district": ward_df.iloc[row,1],
                    "term": ward_df.iloc[row,2],
                    "ward": x
                }
                row_list.append(temp_dict)  
        else:
            temp_dict = {
                "num": ward_df.iloc[row,0], 
                "district": ward_df.iloc[row,1],
                "term": ward_df.iloc[row,2],
                "ward": ward_df.iloc[row,3]
                }
            row_list.append(temp_dict)  
    return row_list

In [27]:
row_list = dewardify(schoolward_df)
#row_list

In [28]:
school_pos_cleaned = school_pos_cleaned.append(row_list)

In [29]:
school_pos_cleaned.ward.value_counts()

1           15
3           14
2           12
7            2
4            2
at-large     2
9            2
5            1
Name: ward, dtype: int64

In [30]:
school_pos_cleaned

,num,district,term,ward
0,4,ALLEGHENY VALLEY,4,NaN
1,4,AVONWORTH,4,NaN
2,4,BALDWIN-WHITEHALL,4,NaN
3,4,BETHEL PARK,4,NaN
4,4,BRENTWOOD,4,NaN
...,...,...,...,...
45,2,WEST ALLEGHENY,4,2
46,1,WOODLAND HILLS,4,1
47,1,WOODLAND HILLS,4,2
48,2,WOODLAND HILLS,4,3


In [31]:
school_pos_cleaned = school_pos_cleaned[school_pos_cleaned.district != "PITTSBURGH"]

## Data Analysis

In [32]:
school_cleaned.head()

,name,party,district,ward
0,Carlos A. Thomas,Democratic,DISTRICT 1,NaN
1,Sylvia C. Wilson,Democratic,DISTRICT 1,NaN
2,Grace Higginbotham,Democratic,DISTRICT 1,NaN
3,Sylvia C. Wilson,Republican,DISTRICT 1,NaN
4,"Lamont Frazier, Jr.",Democratic,DISTRICT 3,NaN


In [33]:
school_cleaned.district.value_counts()

PLUM                    26
MCKEESPORT AREA         24
MOON AREA               22
GATEWAY                 21
SHALER AREA             20
MT LEBANON              20
PINE-RICHLAND           18
AVONWORTH               17
NORTHGATE               16
NORTH ALLEGHENY         16
NORTH HILLS             16
CHARTIERS VALLEY        15
WEST MIFFLIN AREA       14
BETHEL PARK             14
SOUTH ALLEGHENY         12
RIVERVIEW               12
PENN HILLS              12
QUAKER VALLEY           12
WILKINSBURG             11
FOX CHAPEL              11
WEST ALLEGHENY          11
BRENTWOOD               10
DEER LAKES              10
STO-ROX                 10
UPPER SAINT CLAIR       10
CARLYNTON               10
KEYSTONE OAKS            9
WOODLAND HILLS           9
HIGHLANDS                8
HAMPTON                  8
MONTOUR                  8
SOUTH FAYETTE            8
ALLEGHENY VALLEY         8
SOUTH PARK               8
BALDWIN WHITEHALL        8
ELIZABETH FORWARD        8
WEST JEFFERSON HILLS     8
C

In [34]:
school_pos_cleaned.district.value_counts()

CLAIRTON                5
EAST ALLEGHENY          5
STO-ROX                 5
WOODLAND HILLS          4
FOX CHAPEL AREA         4
SHALER AREA             3
PINE-RICHLAND           3
WEST ALLEGHENY          3
HIGHLANDS               3
QUAKER VALLEY           3
KEYSTONE OAKS           3
MONTOUR                 2
STEEL VALLEY            2
CHARTIERS VALLEY        2
SOUTH ALLEGHENY         2
BRENTWOOD               2
NORTHGATE               1
MOUNT LEBANON           1
BALDWIN-WHITEHALL       1
PENN HILLS              1
GATEWAY                 1
WEST JEFFERSON HILLS    1
WILKINSBURG             1
HAMPTON                 1
AVONWORTH               1
DEER LAKES              1
MCKEESPORT AREA         1
CORNELL                 1
DUQUESNE                1
UPPER ST. CLAIR         1
ELIZABETH-FORWARD       1
NORTH HILLS             1
ALLEGHENY VALLEY        1
RIVERVIEW               1
SOUTH PARK              1
SOUTH FAYETTE           1
NORTH ALLEGHENY         1
BETHEL PARK             1
WEST MIFFLIN

In [35]:
no_candidates = []
for dis in school_pos_cleaned.district:
    if dis not in list(school_cleaned.district):
        no_candidates.append(dis)

In [36]:
no_candidates = list(set(no_candidates))
no_candidates = [x.capitalize() for x in no_candidates]
for x in sorted(no_candidates):
    print(x)

Baldwin-whitehall
Elizabeth-forward
Fox chapel area
Mount lebanon
Upper st. clair


# Municipality Data
## Data Cleaning

In [37]:
candidate_df["District Type"].value_counts()

MUNICIPAL         695
SPECPREC          424
SCHOOL            383
WARD              222
SCH/REGION        120
COUNTYWIDE         83
MAGISTERIAL        72
CITY SCHOOLS       15
COUNTY COUNCIL     11
CITY COUNCIL        6
CITYWIDE            4
DISTRICT            2
Name: District Type, dtype: int64

In [38]:
#include
candidate_df[candidate_df["District Type"]=="MUNICIPAL"]["Office Title"].value_counts()

Member of Council             331
Mayor                         112
Tax Collector                  96
Constable                      64
Commissioner                   38
Supervisor                     34
Auditor                         7
Member of Council At-Large      6
Commissioner At-Large           4
Controller                      2
Treasurer                       1
Name: Office Title, dtype: int64

In [39]:
candidate_df[candidate_df["Office Title"]=="Treasurer"]

,First Name,Last Name,Address 1,Address 2,City,Zip,DO,Office Title,Pseudo,District,...,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Unnamed: 20,Office ID
904,Joseph,Senko,NaN,"700 Bower Hill Rd., Apt. 7112",Pittsburgh,15234,1,Treasurer,426,MT LEBANON,...,False,173,1,6,4,1,MUNICIPAL,MUN173,NaN,1650


In [40]:
# do not include
candidate_df[candidate_df["District Type"]=="SPECPREC"]["Office Title"].value_counts()

Judge of Election        243
Inspector of Election    181
Name: Office Title, dtype: int64

In [41]:
# include
candidate_df[candidate_df["District Type"]=="WARD"]["Office Title"].value_counts()

Member of Council    129
Constable             52
Commissioner          41
Name: Office Title, dtype: int64

In [42]:
# do not include
candidate_df[candidate_df["District Type"]=="SCH/REGION"]["Office Title"].value_counts()

School Director    120
Name: Office Title, dtype: int64

In [43]:
# include
candidate_df[candidate_df["District Type"]=="DISTRICT"]

,First Name,Last Name,Address 1,Address 2,City,Zip,DO,Office Title,Pseudo,District,...,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Unnamed: 20,Office ID
1495,John A.,Marshall,NaN,4149 Glasgow Rd.,Valencia,16059,1,Supervisor,1030,Richland 1,...,False,194,0,1,4,1,DISTRICT,Richland1,NaN,2085
1496,Ann M.,Miller,NaN,1028 Applejack Dr.,Gibsonia,15044,1,Supervisor,1031,Richland 2,...,False,194,0,2,4,1,DISTRICT,Richland2,NaN,2085


In [44]:
candidate_df[candidate_df["District Type"]=="WARD"].head()

,First Name,Last Name,Address 1,Address 2,City,Zip,DO,Office Title,Pseudo,District,...,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Unnamed: 20,Office ID
954,Arthur,Buccigrossi III,NaN,971 Glen Hays Drive,Crescent,15046,1,Commissioner,148,CRESCENT WARD 2,...,False,127,2,0,4,1,WARD,WRD1002,NaN,1945
955,Ed,Kissel,NaN,406 Marion St.,Creighton,15030,1,Commissioner,173,EAST DEER WARD 2,...,False,131,2,0,4,1,WARD,WRD1202,NaN,1945
956,Kyle A.,Walk,NaN,609 Oberdick Dr.,McKeesport,15135,1,Commissioner,188,ELIZABETH TOWNSHIP WARD 2,...,False,137,2,0,4,1,WARD,WRD1402,NaN,1945
957,Kevin R.,Cunningham,NaN,2029 Country Club Dr.,McKeesport,15135,1,Commissioner,190,ELIZABETH TOWNSHIP WARD 4,...,False,137,4,1,4,1,WARD,WRD1404,NaN,1945
958,Joshua Michael,Walls,NaN,208 Duncan Station Rd.,McKeesport,15135,1,Commissioner,191,ELIZABETH TOWNSHIP WARD 4,...,False,137,4,2,4,1,WARD,WRD1404,NaN,1945


In [45]:
muni_df = candidate_df[candidate_df["District Type"].isin(["MUNICIPAL","WARD","DISTRICT"])]

In [46]:
#lets not bother w pittsburgh
muni_df = muni_df[muni_df["District"].isin(pgh_districts) == False]

In [47]:
muni_df.head()

,First Name,Last Name,Address 1,Address 2,City,Zip,DO,Office Title,Pseudo,District,...,Need to File,Muni,Ward,District.1,Term,Vote For,District Type,District ID,Unnamed: 20,Office ID
694,David C.,Brown,NaN,227 First Street,Aspinwall,15215,1,Mayor,2,ASPINWALL,...,False,102,0,1,4,1,MUNICIPAL,MUN102,NaN,1340
695,Joseph,Noro,NaN,24 3rd St.,Pittsburgh,15215,1,Mayor,2,ASPINWALL,...,False,102,0,1,4,1,MUNICIPAL,MUN102,NaN,1340
696,Thomas R,"Lloyd, Jr.",NaN,225 South Ohio St.,Pittsburgh,15202,1,Mayor,5,AVALON,...,False,103,1,0,4,1,MUNICIPAL,MUN103,NaN,1340
697,David M.,Depretis,NaN,2957 Meadowvue Dr.,Pittsburgh,15227,1,Mayor,22,BALDWIN BOROUGH,...,False,104,0,12,4,1,MUNICIPAL,MUN104,NaN,1340
698,Kenneth L.,Alvania,NaN,141 Sevin Rd.,Sewickley,15143,1,Mayor,31,BELL ACRES,...,False,106,0,1,4,1,MUNICIPAL,MUN106,NaN,1340


In [48]:
muni_indices = list(muni_df.index)

In [49]:
name = [] #first name + last name
party = [] 
district = [] #District
office = [] #Office Title
ward = [] #Ward
for i in muni_indices:
    row = muni_df[muni_df.index==i]
    name.append(row["First Name"].to_string(index=False) + " " + row["Last Name"].to_string(index=False))
    party.append(row["Party"].to_string(index=False))
    district.append(row["District"].to_string(index=False))
    office.append(row["Office Title"].to_string(index=False))
    ward.append(row["Ward"].to_string(index=False))

In [50]:
muni_cleaned = pd.DataFrame()
muni_cleaned["name"] = name
muni_cleaned["party"] = party
muni_cleaned["district"] = district
muni_cleaned["office"] = office
muni_cleaned["ward"] = ward

In [51]:
muni_cleaned.head()

,name,party,district,office,ward
0,David C. Brown,Democratic,ASPINWALL,Mayor,0
1,Joseph Noro,Republican,ASPINWALL,Mayor,0
2,"Thomas R Lloyd, Jr.",Democratic,AVALON,Mayor,1
3,David M. Depretis,Democratic,BALDWIN BOROUGH,Mayor,0
4,Kenneth L. Alvania,Democratic,BELL ACRES,Mayor,0


In [52]:
muni_cleaned_indices = list(muni_cleaned.index)

In [53]:
# here i'm figuring out if the ward # matches the ward # within the district name
for i in muni_cleaned_indices:
    row = muni_cleaned[muni_cleaned.index==i]
    pts = row.district.to_string(index=False).split(" WARD ")
    if len(pts)>1:
        if int(pts[-1])!=int(row.ward.to_string(index=False)):
            print(pts)
            print(row)
            print()
    

['ROSS', '4']
            name       party     district        office ward
282  Eric Ticjak  Republican  ROSS WARD 4  Commissioner    2

['UPPER ST CLAIR', '4']
                name       party               district        office ward
297  Todd Burlingame  Republican  UPPER ST CLAIR WARD 4  Commissioner    0



In [54]:
# ok so idk wtf is up w those two but i guess i will just move on
muni_cleaned.district = muni_cleaned.district.map(lambda x: x.split(" WARD ")[0])
muni_cleaned= muni_cleaned.replace("Richland 1","RICHLAND")
muni_cleaned= muni_cleaned.replace("Richland 2","RICHLAND")

In [55]:
muni_cleaned.head()

,name,party,district,office,ward
0,David C. Brown,Democratic,ASPINWALL,Mayor,0
1,Joseph Noro,Republican,ASPINWALL,Mayor,0
2,"Thomas R Lloyd, Jr.",Democratic,AVALON,Mayor,1
3,David M. Depretis,Democratic,BALDWIN BOROUGH,Mayor,0
4,Kenneth L. Alvania,Democratic,BELL ACRES,Mayor,0


In [56]:
muni_pos_df = pd.read_csv("munipos.csv")

In [57]:
muni_pos_df.head()

,number,position,term,ward,muni
0,1,TAX COLLECTOR,NaN,NaN,ALEPPO
1,3,COMMISSIONER,4,NaN,ALEPPO
2,1,COMMISSIONER,2,NaN,ALEPPO
3,2,CONSTABLE,NaN,NaN,ALEPPO
4,1,MAYOR,NaN,NaN,ASPINWALL


In [58]:
muni_pos_cleaned = muni_pos_df[muni_pos_df.ward.notna()==False]

In [59]:
dewardify_df = muni_pos_df[muni_pos_df.ward.notna()==True]

In [60]:
dewardify_df.number.value_counts()

3     28
1     24
2     11
4     11
5      3
7      1
9      1
12     1
Name: number, dtype: int64

In [61]:
def dewardify2(ward_df):
    row_list = []
    for row in range(len(ward_df)):
        print(regions)
        num = int(ward_df.iloc[row,0])//len(regions)
        if len(regions)>1:
            for x in regions:
                temp_dict = {
                    "number": num, 
                    "position": ward_df.iloc[row,1],
                    "term": ward_df.iloc[row,2],
                    "ward": x,
                    "muni": ward_df.iloc[row,4]
                }
                row_list.append(temp_dict)  
        else:
            temp_dict = {
                "number": int(ward_df.iloc[row,0]), 
                "position": ward_df.iloc[row,1],
                "term": ward_df.iloc[row,2],
                "ward": ward_df.iloc[row,3],
                "muni": ward_df.iloc[row,4]
                }
            row_list.append(temp_dict)  
    return row_list

In [62]:
muni_pos_cleaned.head()

,number,position,term,ward,muni
0,1,TAX COLLECTOR,NaN,NaN,ALEPPO
1,3,COMMISSIONER,4,NaN,ALEPPO
2,1,COMMISSIONER,2,NaN,ALEPPO
3,2,CONSTABLE,NaN,NaN,ALEPPO
4,1,MAYOR,NaN,NaN,ASPINWALL


In [63]:
muni_pos_cleaned.position.unique()

array(['TAX COLLECTOR', 'COMMISSIONER', 'CONSTABLE', 'MAYOR',
       'MEMBER OF COUNCIL', 'AUDITOR', 'CONTROLLER', 'SUPERVISOR',
       'MEMBERS OF COUNCIL', 'TREASURER'], dtype=object)

In [64]:
muni_pos_cleaned["position"] = muni_pos_cleaned["position"].map(str.lower)
muni_pos_cleaned= muni_pos_cleaned.replace("members of council","member of council")

<ipython-input-64-b115d6eaf588>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  muni_pos_cleaned["position"] = muni_pos_cleaned["position"].map(str.lower)


In [65]:
muni_pos_cleaned.position.unique()

array(['tax collector', 'commissioner', 'constable', 'mayor',
       'member of council', 'auditor', 'controller', 'supervisor',
       'treasurer'], dtype=object)

In [66]:
muni_cleaned.office.unique()

array(['Mayor', 'Controller', 'Tax Collector', 'Treasurer', 'Auditor',
       'Commissioner At-Large', 'Commissioner',
       'Member of Council At-Large', 'Member of Council', 'Supervisor',
       'Constable'], dtype=object)

In [67]:
muni_cleaned["position"] = muni_cleaned.office.map(str.lower)
muni_cleaned = muni_cleaned.drop(columns=["office"])

In [68]:
muni_cleaned.position.unique()

array(['mayor', 'controller', 'tax collector', 'treasurer', 'auditor',
       'commissioner at-large', 'commissioner',
       'member of council at-large', 'member of council', 'supervisor',
       'constable'], dtype=object)

In [69]:
muni_cleaned = muni_cleaned.replace("commissioner at-large","commissioner")
muni_cleaned = muni_cleaned.replace("member of council at-large","member of council")
muni_cleaned.position.unique()

array(['mayor', 'controller', 'tax collector', 'treasurer', 'auditor',
       'commissioner', 'member of council', 'supervisor', 'constable'],
      dtype=object)

In [70]:
sorted(list(muni_cleaned.position.unique())) == sorted(list(muni_pos_cleaned.position.unique()))

True

In [71]:
# scream
sorted(list(muni_pos_cleaned.muni.unique())) == sorted(list(muni_cleaned.district.unique()))

False

In [72]:
def get_bad_names(l1,l2):
    l3 = list(set(l1 + l2))
    bad = []
    for x in l3:
        if x not in l1 and l2:
            bad.append(x)
    return bad

In [73]:
print(sorted(list(muni_pos_cleaned.muni.unique())))
print()
print(sorted(list(muni_cleaned.district.unique())))

['ALEPPO', 'ASPINWALL', 'AVALON', 'BALDWIN BOROUGH', 'BALDWIN TOWNSHIP', 'BELL ACRES', 'BELLEVUE', 'BEN AVON', 'BEN AVON HEIGHTS', 'BETHEL PARK', 'BLAWNOX', 'BRACKENRIDGE', 'BRADDOCK', 'BRADDOCK HILLS', 'BRADDOCK WOODS', 'BRENTWOOD', 'BRIDGEVILLE', 'CARNEGIE', 'CASTLE SHANNON', 'CHALFANT', 'CHESWICK', 'CHURCHILL', 'CLAIRTON', 'COLLIER', 'CORAOPOLIS', 'CRAFTON', 'CRESCENT', 'DORMONT', 'DRAVOSBURG', 'DUQUESNE', 'EAST DEER', 'EAST MCKEESPORT', 'EAST PITTSBURGH', 'EDGEWOOD', 'EDGEWORTH', 'ELIZABETH BOROUGH', 'ELIZABETH TOWNSHIP', 'EMSWORTH', 'ETNA', 'FAWN', 'FINDLAY', 'FOREST HILLS', 'FORWARD', 'FOX CHAPEL', 'FRANKLIN PARK', 'FRAZER', 'GLASSPORT', 'GLEN OSBORNE', 'GLENFIELD', 'GREEN TREE', 'HAMPTON', 'HARMAR', 'HARRISON', 'HAYSVILLE', 'HEIDELBERG', 'HOMESTEAD', 'INDIANA', 'INGRAM', 'JEFFERSON HILLS', 'KENNEDY', 'KILBUCK', 'LEET', 'LEETSDALE', 'LIBERTY', 'LINCOLN', 'MARSHALL', 'MCCANDLESS', 'MCKEES ROCKS', 'MCKEESPORT', 'MILLVALE', 'MONROEVILLE', 'MOON', 'MOUNT LEBANON', 'MOUNT OLIVER', 'MU

In [74]:
get_bad_names(list(muni_pos_cleaned.muni.unique()), list(muni_cleaned.district.unique()))

['MT OLIVER',
 'ROSSLYN FARM',
 'UPPER ST CLAIR',
 'BRADFORD WOODS ',
 'PLEASANT HILL',
 'OSBORNE',
 'OHARA',
 'MT LEBANON']

In [75]:
def fix_name_issues(muni_list):
    # it doesn't work when i do all the stuff in one line idk
    muni_list2 = []
    for x in muni_list:
        temp = x.strip().replace("MT","MOUNT").replace("ST.","ST")
        if temp=="ROSSLYN FARM":
            muni_list2.append("ROSSLYN FARMS")
        elif temp=="PLEASANT HILL":
            muni_list2.append("PLEASANT HILLS")
        elif temp=="O’HARA":
            muni_list2.append("OHARA")
        elif temp=="OSBORNE":
            muni_list2.append("GLEN OSBORNE")
        else:
            muni_list2.append(temp)
    return muni_list2

In [76]:
muni_pos_cleaned["district"] = fix_name_issues(list(muni_pos_cleaned.muni))
muni_pos_cleaned = muni_pos_cleaned.drop(columns=["muni"])

In [77]:
muni_cleaned["district"] = fix_name_issues(list(muni_cleaned.district))

In [78]:
# s c r e a m
sorted(list(muni_pos_cleaned.district.unique())) == sorted(list(muni_cleaned.district.unique()))

False

In [79]:
get_bad_names(list(muni_pos_cleaned.district.unique()), list(muni_cleaned.district.unique()))

['BRADFORD WOODS']

## Data Analysis

In [80]:
muni_cleaned.head()

,name,party,district,ward,position
0,David C. Brown,Democratic,ASPINWALL,0,mayor
1,Joseph Noro,Republican,ASPINWALL,0,mayor
2,"Thomas R Lloyd, Jr.",Democratic,AVALON,1,mayor
3,David M. Depretis,Democratic,BALDWIN BOROUGH,0,mayor
4,Kenneth L. Alvania,Democratic,BELL ACRES,0,mayor


In [81]:
muni_cleaned.groupby("district")["position"].count()

district
ALEPPO               5
ASPINWALL            9
AVALON              12
BALDWIN BOROUGH      9
BALDWIN TOWNSHIP     2
                    ..
WHITE OAK           10
WHITEHALL            9
WILKINS              3
WILKINSBURG         14
WILMERDING           6
Name: position, Length: 125, dtype: int64

In [82]:
muni_cleaned.loc[muni_cleaned.district=="ALEPPO"][muni_cleaned.position=="tax collector"].name

<ipython-input-82-0694beed5e1a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  muni_cleaned.loc[muni_cleaned.district=="ALEPPO"][muni_cleaned.position=="tax collector"].name


114    Casey Cunningham
115    Clayton R. Steup
Name: name, dtype: object

In [83]:
muni_pos_cleaned.head()

,number,position,term,ward,district
0,1,tax collector,NaN,NaN,ALEPPO
1,3,commissioner,4,NaN,ALEPPO
2,1,commissioner,2,NaN,ALEPPO
3,2,constable,NaN,NaN,ALEPPO
4,1,mayor,NaN,NaN,ASPINWALL


In [84]:
muni_cleaned[lambda df: df["district"]=="ALEPPO"][lambda df: df["position"]=="tax collector"]

,name,party,district,ward,position
114,Casey Cunningham,Democratic,ALEPPO,0,tax collector
115,Clayton R. Steup,Republican,ALEPPO,0,tax collector


In [85]:
mc_df = pd.DataFrame()
mc_df = mc_df.append(muni_pos_cleaned)

In [96]:
def get_candidates(pos_df):
    candidates = []
    can_count = []
    parties = []
    
    for index in pos_df.index:
        row = pos_df.loc[[index]]
        
        pos, dis, ward = row["position"].to_string(index=False), row["district"].to_string(index=False), row["ward"].to_string(index=False)
        
        if ward=="NaN":
            subset = muni_cleaned[lambda df: df["district"]==dis][lambda df: df["position"]==pos]
            can_count.append(len(subset))
            if len(subset)>0:
                candidates.append([x.strip() for x in subset.name.to_string(index=False).split("\n")])
                parties.append([x.strip() for x in subset.party.to_string(index=False).split("\n")])
            else:
                candidates.append(["none"])
                parties.append(["N/A"])
        else:
            subset = muni_cleaned[lambda df: df["district"]==dis][lambda df: df["position"]==pos][lambda df: df["ward"]==ward]
            can_count.append(len(subset))
            if len(subset)>0:
                candidates([x.strip() for x in subset.name.to_string(index=False).split("\n")])
                parties.append([x.strip() for x in subset.party.to_string(index=False).split("\n")])
            else:
                candidates.append(["none"])
                parties.append(["N/A"])
    return candidates, can_count, parties

In [97]:
candidates, can_count, party = get_candidates(mc_df)

In [98]:
mc_df["candidates"] = candidates
mc_df["candidate_count"] = can_count
mc_df["party"] = party

In [99]:
# wooooooo
mc_df.head()
mc_df.to_csv("all_muni_positions.csv")

In [100]:
empty_df = mc_df[mc_df.candidate_count==0]
empty_df

,number,position,term,ward,district,candidates,candidate_count,party
3,2,constable,NaN,NaN,ALEPPO,[none],0,[N/A]
7,1,constable,NaN,NaN,ASPINWALL,[none],0,[N/A]
9,1,tax collector,NaN,NaN,AVALON,[none],0,[N/A]
18,1,tax collector,NaN,NaN,BALDWIN TOWNSHIP,[none],0,[N/A]
20,2,constable,NaN,NaN,BALDWIN TOWNSHIP,[none],0,[N/A]
...,...,...,...,...,...,...,...,...
494,1,mayor,NaN,NaN,WHITAKER,[none],0,[N/A]
501,1,constable,NaN,NaN,WHITEHALL,[none],0,[N/A]
505,1,constable,NaN,NaN,WHITE OAK,[none],0,[N/A]
509,2,constable,NaN,NaN,WILKINS,[none],0,[N/A]


In [101]:
empty_df.to_csv("no_candidates.csv")

In [102]:
unopposed_df = mc_df[mc_df.candidate_count==mc_df.number]
unopposed_df


,number,position,term,ward,district,candidates,candidate_count,party
1,3,commissioner,4,NaN,ALEPPO,"[Judith M. Haluka, Daniel M. Darragh, George J...",3,"[Democratic, Republican, Republican]"
5,1,tax collector,NaN,NaN,ASPINWALL,[Dawn E. Eastley],1,[Democratic]
8,1,mayor,NaN,NaN,AVALON,"[Thomas R Lloyd, Jr.]",1,[Democratic]
14,1,mayor,NaN,NaN,BALDWIN BOROUGH,[David M. Depretis],1,[Democratic]
15,1,tax collector,NaN,NaN,BALDWIN BOROUGH,[Gail Dobson Mikush],1,[Democratic]
...,...,...,...,...,...,...,...,...
503,1,tax collector,NaN,NaN,WHITE OAK,[Lisa Pitchford],1,[Democratic]
506,1,tax collector,NaN,NaN,WILKINS,[Michelle Criner],1,[Democratic]
511,1,tax collector,NaN,NaN,WILKINSBURG,[Lisa Ciccarelli],1,[Democratic]
515,1,mayor,NaN,NaN,WILMERDING,[Gregory M. Jakub],1,[Democratic]


In [103]:
unopposed_df.to_csv("unopposed_candidates.csv")

In [104]:
not_enough_candidates_df = mc_df[lambda df: df.candidate_count!=0][mc_df.candidate_count<mc_df.number]
not_enough_candidates_df

<ipython-input-104-7f684ad084ed>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  not_enough_candidates_df = mc_df[lambda df: df.candidate_count!=0][mc_df.candidate_count<mc_df.number]


,number,position,term,ward,district,candidates,candidate_count,party
38,3,member of council,NaN,NaN,BEN AVON HEIGHTS,[Joseph M. Dawley],1,[Democratic]
57,4,member of council,NaN,NaN,BRADDOCK HILLS,"[Anette Nance, Marguerite L. Bolton, Cassandra...",3,"[Democratic, Democratic, Democratic]"
86,4,member of council,NaN,NaN,CHESWICK,"[Frank Meledandri Jr., Kathleen M. Gillard]",2,"[Republican, Republican]"
98,2,constable,NaN,NaN,COLLIER,[Daniel A. Styche],1,[Democratic]
111,2,constable,NaN,NaN,CRESCENT,[Daniel Perciavalle],1,[Democratic]
114,4,member of council,NaN,NaN,DORMONT,"[Jennifer Mazzocco, Alvaro A. Varela, III, Ile...",3,"[Democratic, Democratic, Democratic]"
147,4,member of council,NaN,NaN,ELIZABETH BOROUGH,"[Chad Rager, Dana Cress]",2,"[Democratic, Democratic]"
151,2,constable,NaN,NaN,ELIZABETH TOWNSHIP,[Gary Corey],1,[Republican]
154,4,member of council,NaN,NaN,EMSWORTH,"[Kevin Yurkovich, Daniel Lenz]",2,"[Democratic, Democratic]"
197,4,member of council,NaN,NaN,GLEN OSBORNE,"[John Orndorff, Mary Scalerico]",2,"[Democratic, Democratic]"


In [105]:
not_enough_candidates_df.to_csv("not_enough_candidates.csv")